In [1]:
pacman::p_load(dplyr,ggplot2,tidyverse,patchwork,data.table,lme4,quantreg,bbmle,ggridges)

In [2]:
sessionInfo()

R version 4.1.0 (2021-05-18)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /vortexfs1/home/akrinos/.conda/envs/EUKulele/envs/rversion4/lib/libopenblasp-r0.3.15.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] ggridges_0.5.4    bbmle_1.0.25      quantreg_5.88     SparseM_1.81     
 [5] lme4_1.1-29       Matrix_1.6-1.1    data.table_1.14.8 patchwork_1.1.2  
 [9] forcats_1.0.0     stringr_1.5.0     purrr_1.0.1       readr_2.1.2      
[13] tidyr_1.3.0       tibble_3.

### Read in data from analysis notebook
`../process-all-fcs-data.ipynb`

In [3]:
final_data = data.frame(fread("../data/new_consol_2.csv"))

### Workflow for parameter estimation

1. Fit curve with maximum likelihood
2. Perform optimization on resulting curve to find x-value of maximum to identify thermal optimum
3. Calculate x-intercepts of curves to identify thermal interval

In [4]:
#### FUNCTIONS ####

nbcurve <- function(x,opt,w,a,b){
  res<-a*exp(b*x)*(1-((x-opt)/(w/2))^2)
  res
}

LL1 <- function (y, x, a, b, w, o){
    N = nbcurve(x=x,a=a,b=b,w=w,opt=o)
    N[N<=0]=0.01
    N=N#log(N) # eliminate missing data from loglikelihood
    y=y#log(y)
    return(-sum(dnorm(y,N,log = TRUE))) # the negative log likelihoods: the order of N and y don't matter)
}

strain_color_frame_harriet = data.frame("Strains_full"=c("RCC6856","RCC6071","RCC1212","RCC3963",
                                                   "RCC874","CCMP1280","RCC4567","RCC914","RCC3492",
                                                   "CCMP371","RCC1215","CCMP377","RCC1239",
                                                   "RCC1222","RCC1256","CCMP374","CCMP379","CCMP1516",
                                                   "CCMP2090","CCMP375"),
                                        "Strains"=c("6856","6071","1212","3963",
                                                   "874","1280","4567","914","3492",
                                                   "371","1215","377","1239",
                                                   "1222","1256","374","379","1516",
                                                   "2090","375"),
                                        "Colors"=c("#4443c6","#c688ff","#7d328b","#da1b9d",
                                                  "#f87fa2","#b90033","#ff5755","#c31d0b",
                                                  "#ff6235","#ffc10c","#e4e366","#5a9f00",
                                                  "#60b574","#00b9bf","#0194e3","#C2C95E",
                                                  "#5EC9C9","#000000","#DBDBDB","#01452c"))

return_params <- function(fcm_frame,strain,return_gr_frame=FALSE) {
    if (strain=="6071_old") {
        growth_rates=fcm_frame %>% 
             dplyr::filter(Strain==strain) %>%
             dplyr::mutate(Date=case_when(grepl("/",Date) ~ as.Date(Date,format="%m/%d/%y"),
                                          TRUE ~ as.Date(Date,format="%Y-%m-%d"))) %>%
             dplyr::select(Date,Strain,Position,Temperature,Rep,Transfer,ConcCellmL,GR_Calc_placeholder) %>%
             separate_rows(Position,Rep,Temperature,sep=",") %>%
             dplyr::mutate(Position=case_when((as.numeric(Position)==4)&(Strain==6071)~0,
                                              (as.numeric(Position)==10)&(as.numeric(Strain)==3963)~0,
                                                 TRUE ~ Position),
                          Temperature=case_when((as.numeric(Position)==10)&(as.numeric(Strain)==3963)~10,
                                                 TRUE ~ Temperature)) %>%
             dplyr::filter(GR_Calc_placeholder %in% c("TF","T0")) %>% dplyr::filter((Transfer=="C")|(Position==0)) %>%
             dplyr::mutate(concpivot = paste(ConcCellmL,Date,sep="_")) %>%
             tidyr::pivot_wider(id_cols=c("Strain","Position","Temperature","Rep","Transfer"),
                                names_from = GR_Calc_placeholder, values_from = concpivot) %>%
             tidyr::separate(TF,sep="_",into=c("TF","Date_F"))%>%
             tidyr::separate(T0,sep="_",into=c("T0","Date_0")) %>%
             dplyr::mutate(Date_F = as.Date(Date_F,format="%Y-%m-%d"),
                           Date_0 = as.Date(Date_0,format="%Y-%m-%d"),
                           TF = as.numeric(TF),T0=as.numeric(T0)) %>%
             dplyr::mutate(Duration = lubridate::as.duration(lubridate::interval(as.Date(Date_0,format="%Y-%m-%d"),
                            as.Date(Date_F,format="%Y-%m-%d"))) / lubridate::as.duration(86400) + 1) %>%
             dplyr::mutate(GrowthRate = (log(TF,base=2)-log(T0,base=2))/Duration) %>%
             dplyr::mutate(GrowthRate=case_when(GrowthRate>0 ~ GrowthRate,
                                                TRUE ~ 0.01)) %>%
             dplyr::mutate(ln.r=log(GrowthRate)) %>% dplyr::filter(!((Position==11)&(GrowthRate==0.01)))
    } else if (strain == 374) {
        temperature_correspond_jan20 = data.frame(Position = c(1:20),
                                    Temperature = c(7,8.5,9,11,12,14,15,17,18,19,20,21,21.5,23,23.5,25,
                                                   25.5,26,26.5,28))

        jan2020_data=read.csv("../../data/CombinedTimeSeries_Jan2020_Transfers.csv") %>%
            dplyr::mutate(Date=as.Date(Date,format="%m/%d/%Y"))
        conc_accl=read.csv("../../data/Conc_Oct2020_374_379.csv") %>%
            dplyr::mutate(Date=as.Date(Date,format="%m/%d/%Y"))
        growthrate_379_374=read.csv("../../data/GrowthRateStartEnd_374_379.csv") %>%
            dplyr::mutate(FirstLogDate=as.Date(FirstLogDate,format="%Y-%m-%d"),
                          FinalLogDate=as.Date(FinalLogDate,format="%Y-%m-%d"))
        first_conc=jan2020_data %>% dplyr::mutate(Transfer=as.character(Transfer)) %>%
            dplyr::right_join(growthrate_379_374,by=c("Date"="FirstLogDate",
                                                                              "Strain","Position","Transfer")) %>%
            dplyr::rename(c("FirstConc"="Conc"))
        final_conc=jan2020_data %>% dplyr::mutate(Transfer=as.character(Transfer)) %>%
            dplyr::right_join(growthrate_379_374,by=c("Date"="FinalLogDate",
                                                                                    "Strain","Position","Transfer")) %>%
            dplyr::rename(c("FinalConc"="Conc"))

        first_conc_post=conc_accl %>% dplyr::mutate(Position=as.numeric(Position),Transfer="F") %>% 
            dplyr::right_join(growthrate_379_374,by=c("Date"="FirstLogDate","Strain","Transfer","Position")) %>%
            dplyr::rename(c("FirstConc"="Conc"))
        final_conc_post=conc_accl %>% dplyr::mutate(Position=as.numeric(Position),Transfer="F") %>%
            dplyr::right_join(growthrate_379_374,by=c("Date"="FinalLogDate","Transfer","Strain","Position")) %>%
            dplyr::rename(c("FinalConc"="Conc"))


        t1_grs = first_conc %>%
               dplyr::left_join(final_conc,by=c("Date"="FirstLogDate",
                                                "FinalLogDate"="Date",
                                                "Transfer","Strain",
                                                "Position")) %>%
               dplyr::filter(Transfer==1)%>%
            dplyr::mutate(Duration=lubridate::as.duration(lubridate::interval(as.Date(Date,format="%Y-%m-%d"),
                            as.Date(FinalLogDate,format="%Y-%m-%d"))) / lubridate::as.duration(86400) + 1) %>%
            dplyr::mutate(GrowthRate=(log(FinalConc,base=2)-log(FirstConc,base=2))/Duration) %>%
            dplyr::left_join(temperature_correspond_jan20) %>%
            dplyr::rename(GR_Transfer1=GrowthRate)%>%dplyr::select(Position,Temperature,GR_Transfer1)
        tf_grs = first_conc_post %>%
               dplyr::left_join(final_conc_post, by=c("Date"="FirstLogDate",
                                                "FinalLogDate"="Date",
                                                "Transfer","Strain","Rep",
                                                "Position")) %>%
            dplyr::mutate(Duration=lubridate::as.duration(lubridate::interval(as.Date(Date,format="%Y-%m-%d"),
                            as.Date(FinalLogDate,format="%Y-%m-%d"))) / lubridate::as.duration(86400) + 1) %>%
            dplyr::mutate(GrowthRate=(log(FinalConc,base=2)-log(FirstConc,base=2))/Duration) %>%
            dplyr::left_join(temperature_correspond_jan20) %>%
            dplyr::rename(GR_TransferF=GrowthRate) %>% dplyr::select(Position,Temperature,GR_TransferF)

        growth_rates=t1_grs %>% dplyr::left_join(tf_grs) %>% 
               dplyr::mutate(GrowthRate = dplyr::case_when((Temperature<10)|(Temperature>25) ~ GR_Transfer1,
                                                           TRUE ~ GR_TransferF)) %>%
            dplyr::filter(!is.na(GrowthRate)) %>% dplyr::mutate(Transfer="F") %>%
            dplyr::mutate(GrowthRate=case_when(Temperature==26.5 ~ 0.01,
                                               TRUE ~ GrowthRate))
    } else {
        print(strain)
        growth_rates=fcm_frame %>% 
             dplyr::filter(Strain==strain) %>%
             dplyr::mutate(Date=case_when(grepl("/",Date) ~ as.Date(Date,format="%m/%d/%y"),
                                          TRUE ~ as.Date(Date,format="%Y-%m-%d"))) %>%
             dplyr::select(Date,Strain,Position,Temperature,Rep,Transfer,ConcCellmL,GR_Calc_placeholder) %>%
             separate_rows(Position,Rep,Temperature,sep=",") %>%
             dplyr::mutate(Position=case_when((as.numeric(Position)==4)&(Strain==6071)~0,
                                              (as.numeric(Position)==10)&(as.numeric(Strain)==3963)~0,
                                                 TRUE ~ Position),
                          Temperature=case_when((as.numeric(Position)==10)&(as.numeric(Strain)==3963)~10,
                                                 TRUE ~ Temperature)) %>%
             dplyr::filter(GR_Calc_placeholder %in% c("TF","T0")) %>% dplyr::filter(Transfer=="F") %>%
             dplyr::mutate(concpivot = paste(ConcCellmL,Date,sep="_")) %>%
             tidyr::pivot_wider(id_cols=c("Strain","Position","Temperature","Rep","Transfer"),
                                names_from = GR_Calc_placeholder, values_from = concpivot) %>%
             tidyr::separate(TF,sep="_",into=c("TF","Date_F"))%>%
             tidyr::separate(T0,sep="_",into=c("T0","Date_0")) %>%
             dplyr::mutate(Date_F = as.Date(Date_F,format="%Y-%m-%d"),
                           Date_0 = as.Date(Date_0,format="%Y-%m-%d"),
                           TF = as.numeric(TF),T0=as.numeric(T0)) %>%
             dplyr::mutate(Duration = lubridate::as.duration(lubridate::interval(as.Date(Date_0,format="%Y-%m-%d"),
                            as.Date(Date_F,format="%Y-%m-%d"))) / lubridate::as.duration(86400) + 1) %>%
             dplyr::mutate(GrowthRate = (log(TF,base=2)-log(T0,base=2))/Duration) %>%
             dplyr::mutate(GrowthRate=case_when(GrowthRate>0 ~ GrowthRate,
                                                TRUE ~ 0.01)) %>%
             dplyr::mutate(ln.r=log(GrowthRate)) %>% dplyr::filter(!((Position==11)&(GrowthRate==0.01)))
    }

    color_frame=strain_color_frame_harriet %>% dplyr::filter(Strains==as.character(strain))
    
    ## add extra known data points to drive parameterization
    ## for example, zero growth rates were not calculated, but we add them here since we assume all 
    ## negative growth rates to be zero.
    if (strain==1516) {
        growth_rates = growth_rates %>%
            dplyr::bind_rows(data.frame("GrowthRate"=c(c(rep(0.01,3),rep(0.01,3))),
                                        "Temperature"=as.numeric(c(rep(31.1,3),rep(9,3))),"Transfer"=c("F")))
        print(growth_rates %>% arrange(Temperature) %>% dplyr::select(Temperature,GrowthRate))
    }
    if (strain==2090) {
        growth_rates = growth_rates %>%
            dplyr::bind_rows(data.frame("GrowthRate"=c(c(rep(0.01,9),rep(0.01,9))),
                                        "Temperature"=as.numeric(c(rep(32,9),rep(9,9))),"Transfer"=c("F")))
    }
    
    if (strain==3492) {
        growth_rates = growth_rates %>%
            dplyr::bind_rows(data.frame("GrowthRate"=c(rep(0.01,3)),
                                        "Temperature"=as.numeric(rep(34.4,3)),"Transfer"=c("F")))
    }
    
    if (strain==3963) {
        growth_rates = growth_rates %>% dplyr::filter(Temperature < 27) %>%
            dplyr::bind_rows(data.frame("GrowthRate"=c(0.01),
                                        "Temperature"=as.numeric(30.6),"Transfer"=c("F")))
    }
    
    
    if (strain==375) {
        growth_rates = growth_rates %>%
            dplyr::bind_rows(data.frame("GrowthRate"=as.numeric(c(rep(0.01,3),rep(0.01,3))),
                                        "Temperature"=as.numeric(c(rep(29,3),rep(9,3))),
                                        "Transfer"=c("F")))
    }
    if (strain==1212) {
        growth_rates = growth_rates %>%
            dplyr::bind_rows(data.frame("GrowthRate"=c(rep(0.01,3)),
                                        "Temperature"=as.numeric(rep(12,3)),"Transfer"=c("F")))
    }
    
    
    if (strain==6071) {
        growth_rates = growth_rates %>%
            dplyr::bind_rows(data.frame("GrowthRate"=c(rep(0.01,3),rep(0.01,9)),
                                        "Temperature"=as.numeric(rep(21,3),
                                                                 rep(2,9)),"Transfer"=c("F")))
    }
    
    
    if (strain==374) {
        growth_rates = growth_rates %>%
            dplyr::bind_rows(data.frame("GrowthRate"=c(rep(0.01,9),rep(0.00001,9)),
                                        "Temperature"=as.numeric(c(rep(7,9),rep(6,9))),"Transfer"=c("F")))
    }
    
    
    if (strain==371) {
        growth_rates = growth_rates %>%
            dplyr::bind_rows(data.frame("GrowthRate"=c(rep(0.01,3)),
                                        "Temperature"=as.numeric(rep(32,3)),"Transfer"=c("F")))
    }
    
    if (strain==914) {
        growth_rates = growth_rates %>%
            dplyr::bind_rows(data.frame("GrowthRate"=c(rep(0.01,3),rep(0.01,3)),
                                        "Temperature"=as.numeric(rep(32,3),rep(9,3)),"Transfer"=c("F")))
    }

    
    if (strain==379) {
        growth_rates = growth_rates %>%
            dplyr::bind_rows(data.frame("GrowthRate"=c(rep(0.01,3)),
                                        "Temperature"=as.numeric(rep(8,9)),"Transfer"=c("F")))
    }
    growth_rates = growth_rates %>% dplyr::mutate(Temperature=as.numeric(Temperature))
    
    if (return_gr_frame) {
        return(growth_rates)
    }
    
    ## Create reasonable guesses for thermal optimum & thermal width
    guess_opt = as.numeric(unique((growth_rates %>% 
                            dplyr::filter(GrowthRate==max(GrowthRate,na.rm=T)))$Temperature))
    
    guess_wid = as.numeric(unique((growth_rates %>% dplyr::filter(GrowthRate>0.0) %>%
                            dplyr::filter(Temperature==max(Temperature,na.rm=T)))$Temperature) - 
                           unique((growth_rates %>% dplyr::filter(GrowthRate>0.0) %>%
                            dplyr::filter(Temperature==min(Temperature,na.rm=T)))$Temperature))

    
    x = sort(runif(100,-5,40))
    a = 0.1 # scale param 1
    b = 0.01 # scale param 2
    o_guess = guess_opt # optimum temperature
    w_guess=15
    if ((guess_wid!=0)&(!is.na(guess_wid))) {
        w_guess = guess_wid # thermal niche width
    }
    
    #We want the indepdendet variable to have global scope--
    # i.e. to be available to all functions.
    
    ## Bootstrap additional data points to be used to fit the model
    bootstrapped_points = data.frame()
    for (temp in unique(growth_rates$Temperature)) {
        gr <- mean(as.numeric((growth_rates %>% dplyr::filter(Temperature==temp))$GrowthRate),na.rm=T)
        sd_gr <- sd(as.numeric((growth_rates %>% dplyr::filter(Temperature==temp))$GrowthRate),na.rm=T)
        chisq <- rchisq(1,df=(nrow(growth_rates)-1))
        n <- rnorm(10000, mean = gr, sd = sd_gr*sqrt((nrow(growth_rates)-1)/chisq))
        bootstrapped_points = bootstrapped_points %>%
            dplyr::bind_rows(data.frame("Temperature"=rep(temp,10000),
                                        "GrowthRate"=n)) %>% dplyr::filter(!is.na(GrowthRate)&
                                                                           (GrowthRate>=0))
    }
    
    bootstrapped_points=growth_rates %>% dplyr::arrange(Temperature)
    
    bestguess=10000
    best_m=NA
    print(paste0(bestguess,"Best guess"))
    m1=best_m
    
    m1 = bbmle::mle2(minuslogl = LL1, start = list(a = a, b = b,
                                            o = o_guess,w = w_guess),
              data = list(y=as.numeric(bootstrapped_points$GrowthRate),
                          x=as.numeric(bootstrapped_points$Temperature)),
              control=list(maxit=1000000,abstol=1e-20))
    if (strain==375){
        print(m1)
    }
    print(summary(m1)@coef["b","Estimate"])
    print("^b")

    y = nbcurve(x=x,
            a=summary(m1)@coef["a","Estimate"],
            b=summary(m1)@coef["b","Estimate"],
            w=summary(m1)@coef["w","Estimate"],
            opt=summary(m1)@coef["o","Estimate"])
    plot_frame=data.frame(x=x,
                          y=y,
                          type="Modeled") %>% 
            dplyr::bind_rows(data.frame(x=bootstrapped_points$Temperature,
                             y=bootstrapped_points$GrowthRate,
                             type="Measured")) %>%
            dplyr::mutate(x=as.numeric(x)) %>%
            dplyr::arrange(x)
    plot(x,y, typ='l', col=color_frame$Colors[1], cex.lab = 1.5, cex = 1.5,
    xlab="Temperature", ylab="Growth Rate",
    ylim=c(0,1.5))
    points(growth_rates$Temperature,growth_rates$GrowthRate,col=color_frame$Colors[1],pch=18)
    points(bootstrapped_points$Temperature,bootstrapped_points$GrowthRate,col="black",pch=18)
    title(as.character(strain))
    coef_list = data.frame("a"=summary(m1)@coef["a","Estimate"],
            "b"=summary(m1)@coef["b","Estimate"],
            "w"=summary(m1)@coef["w","Estimate"],
            "opt"=summary(m1)@coef["o","Estimate"],"a_err"=summary(m1)@coef["a","Std. Error"],
            "b_err"=b,
            "w_err"=summary(m1)@coef["w","Std. Error"],
            "opt_err"=summary(m1)@coef["o","Std. Error"],"Strain"=strain)
    return(list(data.frame("Temperature"=x,
                      "ModeledPoints"=y,"Type"="Model",
                      "Strain"=strain) %>% 
           dplyr::bind_rows(data.frame("MeasuredPoints"=growth_rates$GrowthRate,
                                       "Temperature"=as.numeric(growth_rates$Temperature),"Type"="Measure",
                                       "Strain"=strain)),coef_list))
}

par(mfrow = c(3, 2))
all_params=data.frame()
all_params_coef=data.frame()
for (strain in c(374,3963,3492,1516,874,375,2090,379,1212,371,6071,914)) {
    paramslist=return_params(final_data,strain=strain)
    all_params=all_params%>%dplyr::bind_rows(paramslist[[1]])
    all_params_coef=all_params_coef%>%dplyr::bind_rows(paramslist[[2]])
}

Warning message in file(file, "rt"):
“cannot open file '../../data/CombinedTimeSeries_Jan2020_Transfers.csv': No such file or directory”


ERROR: Error in file(file, "rt"): cannot open the connection


In [ ]:
all_params_coef

In [ ]:
revised_df_width =data.frame()
recalculate=TRUE # if this is true, we use optimized Topt rather than the one the eqn spits out.

for (strain_curr in all_params_coef$Strain) {
    opt_val = optimize(nbcurve,interval=c(0,40),maximum=TRUE,
         a=(all_params_coef%>%dplyr::filter(Strain==strain_curr))$a,
         b=(all_params_coef%>%dplyr::filter(Strain==strain_curr))$b,
         w=(all_params_coef%>%dplyr::filter(Strain==strain_curr))$w,
         opt=(all_params_coef%>%dplyr::filter(Strain==strain_curr))$opt)$maximum
    wid_eq_1 = ((all_params_coef%>%dplyr::filter(Strain==strain_curr))$w)/2+
        (all_params_coef%>%dplyr::filter(Strain==strain_curr))$opt
    wid_eq_2 = -((all_params_coef%>%dplyr::filter(Strain==strain_curr))$w)/2+
        (all_params_coef%>%dplyr::filter(Strain==strain_curr))$opt
    if (recalculate) {
        wid_eq_1 = ((all_params_coef%>%dplyr::filter(Strain==strain_curr))$w)/2+
            opt_val
        wid_eq_2 = -((all_params_coef%>%dplyr::filter(Strain==strain_curr))$w)/2+
            opt_val
    }
    revised_df_width = revised_df_width %>%
        dplyr::bind_rows(data.frame(Strain=strain_curr,Revised_wid=abs(wid_eq_2-wid_eq_1)))
}

## Read in data and recalculate parameters from Anderson et al. (2021)

In [ ]:
## Norberg curve parameterization
nbcurve <- function(x,opt,w,a,b){
  res<-a*exp(b*x)*(1-((x-opt)/(w/2))^2)
  res
}

# Read in data
derived_traits<-read.csv("../../data/PFT_thermal_response/data/derived_traits.csv")
gr_fr<-read.csv("../../data/PFT_thermal_response/data/growth_rates.csv")
derived_traits_merge<-merge(derived_traits,gr_fr,by="isolate.code")

plot(seq(-10,30,by=1),nbcurve(seq(-10,30,by=1),
                              derived_traits[1,"mu.c.opt.list"],
                              derived_traits[1,"mu.wlist"],
                              derived_traits[1,"mu.alist"],
                              derived_traits[1,"mu.blist"]))
recalculate=FALSE
revised_df=data.frame()
for (curr in c(1:nrow(derived_traits))) {
  strain_curr=paste(derived_traits[curr,"genus"],
                    derived_traits[curr,"species"],
               derived_traits[curr,"strain"],sep="_")
  group_curr=derived_traits[curr,"group"]
  if (length(group_curr)==0) {
    group_curr="unknown"
  }
  a=derived_traits[curr,"mu.alist"]
  b=derived_traits[curr,"mu.blist"]
  w=derived_traits[curr,"mu.wlist"]
  opt=derived_traits[curr,"mu.c.opt.list"]
  opt_val = optimize(nbcurve,interval=c(-20,40),maximum=TRUE,
                     a=derived_traits[curr,"mu.alist"],
                     b=derived_traits[curr,"mu.blist"],
                     w=derived_traits[curr,"mu.wlist"],
                     opt=opt)$maximum
  if (abs(opt_val) > 100) {
    next # these original parameterizations were bad.
  }
  wid_eq_1 = derived_traits[curr,"mu.wlist"]/2+
    opt
  wid_eq_2 = -derived_traits[curr,"mu.wlist"]/2+
    opt
  tol_gr=0.01 ## we don't want to say the width is valid when GR<this
  test_temp=wid_eq_1
  while ((nbcurve(test_temp,opt,w,a,b) < tol_gr)) {
    test_temp=test_temp+sign(test_temp)*-0.001
  }
  wid_eq_1=test_temp
  test_temp=wid_eq_2
  while ((nbcurve(test_temp,opt,w,a,b) < tol_gr)) {
    test_temp=test_temp+sign(test_temp-wid_eq_1)*-0.001
  }
  wid_eq_2=test_temp
  
  if (recalculate) {
    wid_eq_1 = (derived_traits[curr,"mu.wlist"])/2+
      opt_val
    wid_eq_2 = -(derived_traits[curr,"mu.wlist"])/2+
      opt_val
  }
  tolerance_val=nbcurve(opt_val,opt,w,a,b)*0.8
  temps_test=seq(from = wid_eq_2+sign(wid_eq_2)*wid_eq_2*0.5, 
                 to = wid_eq_1+sign(wid_eq_2)*wid_eq_2*0.5, by = 0.01)
  window_1 = -100
  window_2 = -100
  print(paste0(tolerance_val,"; window",window_1," to ",window_2,"; strain:",strain_curr))
  for (temp in temps_test) {
    if ((window_1<0)&(window_2<0)&(nbcurve(temp,opt,w,a,b) >= tolerance_val)) {
      window_1 = temp
    }else if ((window_2<0)&(window_1>0)&(nbcurve(temp,opt,w,a,b) <= tolerance_val)){
      window_2=temp
    }
  }
  if ((window_1==-100)|(window_2==-100)){
    print("broken")
    break
  }
  if (abs(wid_eq_2-wid_eq_1) > 50) {
    print("broken")
    #break
  }
  revised_df = rbind(revised_df,
                     data.frame(Strain=strain_curr,Revised_opt=opt_val,
                                Group=group_curr,
                                Revised_wid=abs(wid_eq_2-wid_eq_1),
                                Revised_plateau=abs(window_1-window_2),
                                low_temp=window_1,high_temp=window_2,
                                max_GR=nbcurve(opt_val,opt,w,a,b),
                                t_opt=opt_val,a=a,b=b,w=w))
}
# calculate plateau parameter for input data
revised_df$left_plateau=revised_df$t_opt-revised_df$low_temp
revised_df$right_plateau=revised_df$high_temp-revised_df$t_opt

par(mfrow=c(2,2))
for (group in unique(revised_df$Group)) {
  
  plot(x=revised_df[!is.na(revised_df$right_plateau)&
                      !is.na(revised_df$Strain)&(revised_df$Group==group),"left_plateau"],
       y=revised_df[!is.na(revised_df$right_plateau)&
                      !is.na(revised_df$Strain)&(revised_df$Group==group),"right_plateau"],
       pch=20,cex=3,
       col=revised_df[!is.na(revised_df$right_plateau)&
                        !is.na(revised_df$Strain)&(revised_df$Group==group),"Color"],
       ylab="right plateau",
       xlab="left plateau")
  lines(seq(0,15,by=1),seq(0,15,by=1),lty="dotted")
}

plot(seq(wid_eq_2,wid_eq_1,by=1),nbcurve(seq(wid_eq_2,wid_eq_1,by=1),
                              opt,
                              derived_traits[curr,"mu.wlist"],
                              derived_traits[curr,"mu.alist"],
                              derived_traits[curr,"mu.blist"]))


color_frame=data.frame("Group"=c("diatoms","coccolithophores",
                                 "dinoflagellates","cyanobacteria"),
                       "Color"=c("red","orange","green","blue"))
revised_df=merge(revised_df,color_frame)
plot(x=revised_df[!is.na(revised_df$Revised_plateau)&
                    !is.na(revised_df$Strain),"t_opt"],
     y=revised_df[!is.na(revised_df$Revised_plateau)&
                    !is.na(revised_df$Strain),"max_GR"],
     pch=20,cex=3,
     col=revised_df[!is.na(revised_df$Revised_plateau)&
                      !is.na(revised_df$Strain),"Color"],
     ylab="Revised range of temps w/ 80% max GR",
     xlab="Revised thermal width of nonzero GR temps")
for (group in unique(revised_df$Group)) {
  plot(x=revised_df[!is.na(revised_df$Revised_plateau)&
                      !is.na(revised_df$Strain)&
                      revised_df$Group==group,"t_opt"],
       y=revised_df[!is.na(revised_df$Revised_plateau)&
                      !is.na(revised_df$Strain)&
                      revised_df$Group==group,"max_GR"],
       pch=20,
       col=revised_df[!is.na(revised_df$Revised_plateau)&
                        !is.na(revised_df$Strain)&
                        revised_df$Group==group,"Color"],
       ylab="Revised range of temps w/ 80% max GR",
       xlab="Revised thermal width of nonzero GR temps")
  cocco_model=summary(lm(formula = max_GR~t_opt+t_opt*t_opt,
                         data = revised_df[revised_df$Group==group,]))
  cocco_intercept=data.frame(cocco_model$coefficients)["(Intercept)","Estimate"]
  cocco_slope=data.frame(cocco_model$coefficients)["Revised_wid","Estimate"]
  lines(seq(0,60,by=1),seq(0,60,by=1)*cocco_slope+cocco_intercept,
        lty="dotted")
}

plot(x=revised_df[!is.na(revised_df$Revised_plateau)&
                       !is.na(revised_df$Strain),"Revised_wid"],
     y=revised_df[!is.na(revised_df$Revised_plateau)&
                       !is.na(revised_df$Strain),"Revised_plateau"],
     pch=20,cex=3,
     col=revised_df[!is.na(revised_df$Revised_plateau)&
                    !is.na(revised_df$Strain),"Color"],
     ylab="Revised range of temps w/ 80% max GR",
     xlab="Revised thermal width of nonzero GR temps")
boxplot(x=factor(revised_df[!is.na(revised_df$Revised_plateau)&
                       !is.na(revised_df$Strain),"Strain"]),
        y=revised_df[!is.na(revised_df$Revised_plateau)&
                       !is.na(revised_df$Strain),"Revised_plateau"])
boxplot(Revised_plateau ~ Group, data = revised_df, 
        col = unique(revised_df$Color),pch=20,
        xlab="",ylab="Range (°C) of temperatures within 80% of max GR")
lines(y=rep(5,9),x=c(0:8),lty="dotted")
points(x=jitter(as.numeric(factor(revised_df$Group))),
       y=revised_df$Revised_plateau,col=revised_df$Color,pch=20)
boxplot(low_temp ~ Group, data = revised_df, col = "bisque")
boxplot(high_temp ~ Group, data = revised_df, col = "bisque")

boxplot(Revised_wid ~ Group, data = revised_df, 
        col = unique(revised_df$Color),pch=20,
        xlab="",ylab="Range (°C) of temperatures within 80% of max GR")
lines(y=rep(5,9),x=c(0:8),lty="dotted")
points(x=jitter(as.numeric(factor(revised_df$Group))),
       y=revised_df$Revised_plateau,col=revised_df$Color,pch=20)
boxplot(low_temp ~ Group, data = revised_df, col = "bisque")
boxplot(high_temp ~ Group, data = revised_df, col = "bisque")

summary(lm(formula = Revised_plateau~Revised_wid,
   data = revised_df[revised_df$Group=="coccolithophores",]))
summary(lm(formula = Revised_plateau~Revised_wid,
           data = revised_df[revised_df$Group=="diatoms",]))
summary(lm(formula = Revised_plateau~Revised_wid,
           data = revised_df[revised_df$Group=="dinoflagellates",]))
plot(x=revised_df$low_temp,y=revised_df$Revised_plateau,
     col=revised_df$Color)

plot(x=revised_df[!is.na(revised_df$Revised_plateau)&
                    !is.na(revised_df$Strain)&
                    (revised_df$Group=="coccolithophores"),"Revised_wid"],
     y=revised_df[!is.na(revised_df$Revised_plateau)&
                    !is.na(revised_df$Strain)&
                    (revised_df$Group=="coccolithophores"),"Revised_plateau"],
     pch=20,cex=3,
     col=revised_df[!is.na(revised_df$Revised_plateau)&
                    !is.na(revised_df$Strain)&
                    (revised_df$Group=="coccolithophores"),"Color"],
     ylab="Revised range of temps w/ 80% max GR",
     xlab="Revised thermal width of nonzero GR temps")


for (group in unique(revised_df$Group)) {
  plot(x=revised_df[!is.na(revised_df$Revised_plateau)&
                      !is.na(revised_df$Strain)&
                      revised_df$Group==group,"Revised_wid"],
       y=revised_df[!is.na(revised_df$Revised_plateau)&
                      !is.na(revised_df$Strain)&
                      revised_df$Group==group,"Revised_plateau"],
       pch=20,
       col=revised_df[!is.na(revised_df$Revised_plateau)&
                        !is.na(revised_df$Strain)&
                        revised_df$Group==group,"Color"],
       ylab="Revised range of temps w/ 80% max GR",
       xlab="Revised thermal width of nonzero GR temps")
  cocco_model=summary(lm(formula = Revised_plateau~Revised_wid,
                         data = revised_df[revised_df$Group==group,]))
  cocco_intercept=data.frame(cocco_model$coefficients)["(Intercept)","Estimate"]
  cocco_slope=data.frame(cocco_model$coefficients)["Revised_wid","Estimate"]
  lines(seq(0,60,by=1),seq(0,60,by=1)*cocco_slope+cocco_intercept,
        lty="dotted")
}

plot(y=as.numeric(revised_df$Revised_plateau),x=factor(revised_df$Group),
        col="blue")
par(mfrow=c(2,2))
for (group in unique(revised_df$Group)) {
  hist(x=revised_df[revised_df$Group==group,"Revised_plateau"],
       col =revised_df[revised_df$Group==group,"Color"],xlab="Plateau param",
       main=group,xlim=c(0,30))

}

par(mfrow=c(2,2))
for (group in unique(revised_df$Group)) {
  plot(x=revised_df[!is.na(revised_df$Revised_plateau)&
                      !is.na(revised_df$Strain)&
                      revised_df$Group==group,"t_opt"],
       y=revised_df[!is.na(revised_df$Revised_plateau)&
                      !is.na(revised_df$Strain)&
                      revised_df$Group==group,"Revised_plateau"],
       pch=20,
       col=revised_df[!is.na(revised_df$Revised_plateau)&
                        !is.na(revised_df$Strain)&
                        revised_df$Group==group,"Color"],
       ylab="thermal width of 80% of max growth rate",
       xlab="Thermal optimum (°C)",ylim=c(0,26),xlim=c(0,35))
  dev.hold()
  cocco_model=summary(lm(formula = Revised_plateau~t_opt,
                         data = revised_df[revised_df$Group==group,]))
  cocco_intercept=data.frame(cocco_model$coefficients)["(Intercept)","Estimate"]
  cocco_slope=data.frame(cocco_model$coefficients)["t_opt","Estimate"]
  lines(seq(0,60,by=1),seq(0,60,by=1)*cocco_slope+cocco_intercept,
        lty="dotted")
}
par(mfrow=c(1,2))

boxplot(Revised_plateau ~ Group, data = revised_df, 
        col = unique(revised_df$Color),pch=20,
        xlab="",ylab="thermal width (°C) of 80% of max growth rate")
lines(y=rep(5,9),x=c(0:8),lty="dotted")
points(x=jitter(as.numeric(factor(revised_df$Group))),
       y=revised_df$Revised_plateau,col=revised_df$Color,pch=20)

plot(x=revised_df[!is.na(revised_df$Revised_plateau)&
                    !is.na(revised_df$Strain),"t_opt"],
     y=revised_df[!is.na(revised_df$Revised_plateau)&
                    !is.na(revised_df$Strain),"Revised_plateau"],
     pch=20,
     col=revised_df[!is.na(revised_df$Revised_plateau)&
                      !is.na(revised_df$Strain),"Color"],
     ylab="thermal width (°C) of 80% of max growth rate",
     xlab="Thermal optimum (°C)",ylim=c(0,26),xlim=c(0,35))
for (group in c("diatoms","cyanobacteria")) {#unique(revised_df$Group)) {
  dev.hold()
  cocco_model=summary(lm(formula = Revised_plateau~t_opt,
                         data = revised_df[revised_df$Group==group,]))
  print(group)
  print(cocco_model)
  cocco_intercept=data.frame(cocco_model$coefficients)["(Intercept)","Estimate"]
  cocco_slope=data.frame(cocco_model$coefficients)["t_opt","Estimate"]
  lines(seq(-10,60,by=1),seq(-10,60,by=1)*cocco_slope+cocco_intercept,
        lty="dotted",col=unique(revised_df[revised_df$Group==group,"Color"]),
        lwd=2)
}
cocco_model=summary(lm(formula = Revised_plateau~t_opt,
                       data = revised_df))


## Recalculate parameters from van Dassow et al.

In [ ]:
vandassow=read.csv("../../data/Table4_vanDassow.csv")
vandassow=vandassow%>%dplyr::select(Experiment,Strain,Species,
                                    Haplogroup,Growth_Rate_8_deg,Growth_Rate_12_deg,
                                    Growth_Rate_15_deg,Growth_Rate_18_deg,Growth_Rate_21_deg,
                                    Growth_Rate_24_deg,Growth_Rate_27_deg) %>%
    dplyr::mutate_at(colnames(.)[grepl("Growth_Rate",colnames(.))],as.character) %>%
    tidyr::pivot_longer(cols=starts_with("Growth_Rate"),names_to="Temp",values_to="GrowthRate") %>%
    tidyr::separate(Temp,sep="_",into=c("Sep1","Sep2","Temperature","deg")) %>%
    dplyr::mutate(Temperature=as.numeric(Temperature),
                  GrowthRate=as.numeric(trimws(as.character(GrowthRate))))

In [ ]:
## recalculate van Dassow parameters


LL1 <- function (y, x, a, b, w, o){
    N = nbcurve(x=x,a=a,b=b,w=w,opt=o)
    N[N<=0]=0.01
    N=N
    y=y
    return(-sum(dnorm(y,N,log = TRUE))) # the negative log likelihoods: the order of N and y don't matter)
}

nbcurve <- function(x,opt,w,a,b){
  res<-a*exp(b*x)*(1-((x-opt)/(w/2))^2)
  res
}

LL1 <- function (y, x, a, b, w, o){
    N = nbcurve(x=x,a=a,b=b,w=w,opt=o)
    N[N<=0]=0.01
    N=N
    y=y
    return(-sum(dnorm(y,N,log = TRUE))) # the negative log likelihoods: the order of N and y don't matter)
}

vandassowparam <- function(vandassow,strain) {
    a = 0.05
    b = 0.01
    x=seq(-5,40,1)
    bootstrapped_points=vandassow%>%dplyr::filter(Strain==strain) %>%
        dplyr::mutate(GrowthRate=case_when(GrowthRate==0~0.01,
                                           TRUE~GrowthRate)) %>%
        dplyr::filter(!is.na(GrowthRate))
    
    print(bootstrapped_points)
    guess_opt = max(as.numeric(unique((bootstrapped_points %>% 
                            dplyr::filter(GrowthRate==max(GrowthRate,na.rm=T)))$Temperature)))
    
    guess_wid = max(as.numeric(unique((bootstrapped_points %>% dplyr::filter(GrowthRate>0.0) %>%
                            dplyr::filter(Temperature==max(Temperature,na.rm=T)))$Temperature) - 
                           unique((bootstrapped_points %>% dplyr::filter(GrowthRate>0.0) %>%
                            dplyr::filter(Temperature==min(Temperature,na.rm=T)))$Temperature)))
    o_guess = guess_opt # optimum temperature
    w_guess=15
    if (guess_wid!=0) {
        w_guess = guess_wid # thermal niche width
    }
    
    print(list(a = a, b = b,
                                            o = o_guess,w = w_guess))
    m1 = bbmle::mle2(minuslogl = LL1, start = list(a = a, b = b,
                                                   w = w_guess,
                                                   o = o_guess),
              data = list(y=as.numeric(bootstrapped_points$GrowthRate),
                          x=as.numeric(bootstrapped_points$Temperature)),
              control=list(maxit=1000000,abstol=1e-20))
    print(summary(m1)@coef["b","Estimate"])
    print("^b")

    y = nbcurve(x=x,
            a=summary(m1)@coef["a","Estimate"],
            b=summary(m1)@coef["b","Estimate"],
            w=summary(m1)@coef["w","Estimate"],
            opt=summary(m1)@coef["o","Estimate"])
    plot_frame=data.frame(x=x,
                          y=y,
                          type="Modeled") %>% 
            dplyr::bind_rows(data.frame(x=bootstrapped_points$Temperature,
                             y=bootstrapped_points$GrowthRate,
                             type="Measured")) %>%
            dplyr::mutate(x=as.numeric(x)) %>%
            dplyr::arrange(x)
    print(y)
    print("^y")
    plot(x,y, typ='l', col="black", cex.lab = 1.5, cex = 1.5,
    xlab="Temperature", ylab="Growth Rate",
    ylim=c(0,1.5))
    points(bootstrapped_points$Temperature,bootstrapped_points$GrowthRate,col="black",pch=18)
    title(as.character(strain))
    coef_list = data.frame("a"=summary(m1)@coef["a","Estimate"],
            "b"=summary(m1)@coef["b","Estimate"],
            "w"=summary(m1)@coef["w","Estimate"],
            "opt"=summary(m1)@coef["o","Estimate"],
            "a_err"=summary(m1)@coef["a","Std. Error"],
            "b_err"=b,
            "w_err"=summary(m1)@coef["w","Std. Error"],
            "opt_err"=summary(m1)@coef["o","Std. Error"],"Strain"=strain)
    return(coef_list)
    return(list(data.frame("Temperature"=x,
                      "ModeledPoints"=y,"Type"="Model",
                      "Strain"=strain) %>% 
           dplyr::bind_rows(data.frame("MeasuredPoints"=bootstrapped_points$GrowthRate,
                                       "Temperature"=as.numeric(bootstrapped_points$Temperature),"Type"="Measure",
                                       "Strain"=strain)),coef_list))
}
all_strains=data.frame()
for (strain in unique(vandassow$Strain)) {
    all_strains=all_strains %>% dplyr::bind_rows(vandassowparam(vandassow,strain))
}

recalculate=FALSE
all_strains=all_strains%>%dplyr::filter(!is.na(opt))
revised_df_vanDassow=data.frame()
for (curr in c(1:nrow(all_strains))) {
  strain_curr=all_strains$Strain[curr]
  group_curr="coccolithophores"
  if (length(group_curr)==0) {
    group_curr="unknown"
  }
  a=all_strains[curr,"a"]
  b=all_strains[curr,"b"]
  w=all_strains[curr,"w"]
  opt=all_strains[curr,"opt"]
  opt_val = optimize(nbcurve,interval=c(-20,40),maximum=TRUE,
                     a=a,
                     b=b,
                     w=w,
                     opt=opt)$maximum
  if (abs(opt_val) > 100) {
    next # these original parameterizations were bad.
  }
  wid_eq_1 = all_strains[curr,"w"]/2+
    opt
  wid_eq_2 = -all_strains[curr,"w"]/2+
    opt
  tol_gr=0.01 ## we don't want to say the width is valid when GR<this
  test_temp=wid_eq_1
  while ((nbcurve(test_temp,opt,w,a,b) < tol_gr)) {
    test_temp=test_temp+sign(test_temp)*-0.001
  }
  wid_eq_1=test_temp
  test_temp=wid_eq_2
  while ((nbcurve(test_temp,opt,w,a,b) < tol_gr)) {
    test_temp=test_temp+sign(test_temp-wid_eq_1)*-0.001
  }
  wid_eq_2=test_temp
  
  if (recalculate) {
    wid_eq_1 = (all_strains[curr,"w"])/2+
      opt_val
    wid_eq_2 = -(all_strains[curr,"w"])/2+
      opt_val
  }
  tolerance_val=nbcurve(opt_val,opt,w,a,b)*0.8
  temps_test=seq(from = wid_eq_2+sign(wid_eq_2)*wid_eq_2*0.5, 
                 to = wid_eq_1+sign(wid_eq_2)*wid_eq_2*0.5, by = 0.01)
  window_1 = -100
  window_2 = -100
  print(paste0(tolerance_val,"; window",window_1," to ",window_2,"; strain:",strain_curr))
  for (temp in temps_test) {
    if ((window_1<0)&(window_2<0)&(nbcurve(temp,opt,w,a,b) >= tolerance_val)) {
      window_1 = temp
    }else if ((window_2<0)&(window_1>0)&(nbcurve(temp,opt,w,a,b) <= tolerance_val)){
      window_2=temp
    }
  }
  if ((window_1==-100)|(window_2==-100)){
    print("broken")
    break
  }
  if (abs(wid_eq_2-wid_eq_1) > 50) {
    print("broken")
    #break
  }
  revised_df_vanDassow = rbind(revised_df_vanDassow,
                     data.frame(Strain=strain_curr,Revised_opt=opt_val,
                                Group=group_curr,
                                Revised_wid=abs(wid_eq_2-wid_eq_1),
                                Revised_plateau=abs(window_1-window_2),
                                low_temp=window_1,high_temp=window_2,
                                max_GR=nbcurve(opt_val,opt,w,a,b),
                                t_opt=opt_val,a=a,b=b,w=w))
}
revised_df_vanDassow$left_plateau=revised_df_vanDassow$t_opt-revised_df_vanDassow$low_temp
revised_df_vanDassow$right_plateau=revised_df_vanDassow$high_temp-revised_df_vanDassow$t_opt

In [ ]:
nbcurve(x = opt_val,a=(all_params_coef%>%dplyr::filter(Strain==strain_curr))$a,
         b=(all_params_coef%>%dplyr::filter(Strain==strain_curr))$b,
         w=(all_params_coef%>%dplyr::filter(Strain==strain_curr))$w,
         opt=(all_params_coef%>%dplyr::filter(Strain==strain_curr))$opt)

In [ ]:
revised_df_2 =data.frame()
for (strain_curr in all_params_coef$Strain) {
    opt_val = optimize(nbcurve,interval=c(0,40),maximum=TRUE,
         a=(all_params_coef%>%dplyr::filter(Strain==strain_curr))$a,
         b=(all_params_coef%>%dplyr::filter(Strain==strain_curr))$b,
         w=(all_params_coef%>%dplyr::filter(Strain==strain_curr))$w,
         opt=(all_params_coef%>%dplyr::filter(Strain==strain_curr))$opt)$maximum
    gr_at_opt=nbcurve(x = opt_val,a=(all_params_coef%>%dplyr::filter(Strain==strain_curr))$a,
         b=(all_params_coef%>%dplyr::filter(Strain==strain_curr))$b,
         w=(all_params_coef%>%dplyr::filter(Strain==strain_curr))$w,
         opt=(all_params_coef%>%dplyr::filter(Strain==strain_curr))$opt)
    revised_df_2 = revised_df_2 %>%
        dplyr::bind_rows(data.frame(Strain=strain_curr,Revised_opt=opt_val,
                                    Revised_gr_at_opt = gr_at_opt))
}

In [ ]:
revised_df_2

In [ ]:
eppley_comp = ggplot(all_params_coef%>%dplyr::left_join(revised_df_2 %>% 
                                                        dplyr::mutate(Strain=as.numeric(as.character(Strain)))) %>%
          dplyr::left_join(all_params %>% dplyr::group_by(Strain) %>% 
                           dplyr::filter(MeasuredPoints==max(MeasuredPoints,na.rm=T))) %>%
          dplyr::mutate(Strain=factor(Strain,levels=unique(Strain))))+ 
    geom_point(data=revised_df%>%dplyr::filter(Group=="coccolithophores"),
               aes(x=t_opt,y=max_GR,col="Anderson et al. (2021)"),pch=18)+
    geom_point(data=revised_df_vanDassow,
               aes(x=t_opt,y=max_GR,col="van Dassow et al. (2021)"),pch=18)+
    geom_point(aes(x = Revised_opt, y =MeasuredPoints,fill=Strain),size=3,color="black",
               pch=21) +
    geom_line(data=data.frame(x=c(0:30),y=0.741*exp(0.035*c(0:30))),
              mapping=aes(x=x,y=y),linetype="dotdash") + 
    theme_bw(base_size=12) + 
    ylab("Estimated maximum growth rate (1/day)") + xlab("Estimated thermal optimum (°C)") + 
    scale_fill_manual(breaks=as.character((strain_color_frame_harriet %>% dplyr::filter(Strains %in% all_params$Strain))$Strains),
                                        limits=as.character((strain_color_frame_harriet %>% dplyr::filter(Strains %in% all_params$Strain))$Strains),
                                        values=(strain_color_frame_harriet %>% 
                                                dplyr::filter(Strains %in% all_params$Strain))$Colors) +
    scale_color_manual(values=c("light gray","brown"),name="") +
    scale_size(name="Thermal width")
eppley_comp

In [ ]:
revised_df_plateau =data.frame() ## range of temperatures within 10% of max GR. 
# we set res to be 10% of opt on either side.
recalculate=TRUE # if this is true, we use optimized Topt rather than the one the eqn spits out.

nbcurve <- function(x,opt,w,a,b){
  res<-a*exp(b*x)*(1-((x-opt)/(w/2))^2)
  res
}

for (strain_curr in all_params_coef$Strain) {
    a=(all_params_coef%>%dplyr::filter(Strain==strain_curr))$a
    b=(all_params_coef%>%dplyr::filter(Strain==strain_curr))$b
    w=(all_params_coef%>%dplyr::filter(Strain==strain_curr))$w
    opt=(all_params_coef%>%dplyr::filter(Strain==strain_curr))$opt
    opt_val = optimize(nbcurve,interval=c(0,40),maximum=TRUE,
         a=a,
         b=b,
         w=w,
         opt=(all_params_coef%>%dplyr::filter(Strain==strain_curr))$opt)$maximum
    wid_eq_1 = ((all_params_coef%>%dplyr::filter(Strain==strain_curr))$w)/2+
        opt_val
    wid_eq_2 = -((all_params_coef%>%dplyr::filter(Strain==strain_curr))$w)/2+
        opt_val
    tolerance_val=nbcurve(opt_val,opt,w,a,b)*0.8
    temps_test=seq(from = wid_eq_2+sign(wid_eq_2)*wid_eq_2*0.5, 
                   to = wid_eq_1+sign(wid_eq_2)*wid_eq_2*0.5, by = 0.01)
    window_1 = -100
    window_2 = -100
    print(paste0(tolerance_val,"; window",window_1," to ",window_2,"; strain:",strain_curr))
    for (temp in temps_test) {
        if ((window_1<0)&(window_2<0)&(nbcurve(temp,opt,w,a,b) >= tolerance_val)) {
            window_1 = temp
        }else if ((window_2<0)&(window_1>0)&(nbcurve(temp,opt,w,a,b) <= tolerance_val)){
            window_2=temp
        }
    }
    revised_df_plateau = revised_df_plateau %>%
        dplyr::bind_rows(data.frame(Strain=strain_curr,Revised_plateau=abs(window_1-window_2),
                                    low_temp=window_1,high_temp=window_2,topt=opt_val,
                                    gr_opt=nbcurve(opt_val,opt,w,a,b)))
}
revised_df_plateau$left_plateau=revised_df_plateau$topt-revised_df_plateau$low_temp
revised_df_plateau$right_plateau=revised_df_plateau$high_temp-revised_df_plateau$topt



In [ ]:
max(revised_df_plateau$Revised_plateau)-min(revised_df_plateau$Revised_plateau)

In [ ]:
ggplot(revised_df_plateau) + geom_point(aes(x=left_plateau,y=right_plateau,fill=factor(Strain)),pch=21,size=4) + 
    geom_abline(aes(intercept=0,slope=1),linetype="dotdash")+ theme_bw(base_size=12)+ 
    scale_fill_manual(breaks=as.character((strain_color_frame_harriet%>%
                                            dplyr::filter(Strains%in%all_params_coef$Strain))$Strains),
                                        limits=as.character((strain_color_frame_harriet%>%
                                            dplyr::filter(Strains%in%all_params_coef$Strain))$Strains),
                                        values=(strain_color_frame_harriet%>%
                                            dplyr::filter(Strains%in%all_params_coef$Strain))$Colors,
                       name="Strain") +
    geom_point(data=revised_df%>%dplyr::filter(Group=="coccolithophores"),
               aes(x=left_plateau,y=right_plateau,color="Anderson et al. 2021")) + 
    scale_color_manual(limits=c("Anderson et al. 2021"),values="gray",name="") + 
    xlab("Range (°C) of temperatures\nbelow optimum within 80% of max GR")+ 
    ylab("Range (°C) of temperatures\nabove optimum within 80% of max GR")

ggsave("../../figures/supplemental/plateau_plot.png")

In [ ]:
built_gr_df = data.frame()
for (curr in 1:nrow(all_params_coef)) {
    built_gr_df=built_gr_df %>% dplyr::bind_rows(data.frame(Strain=all_params_coef$Strain[curr],
                                                            Temperature = seq(0,35,by=0.01),
                                                            ModeledPoints = nbcurve(seq(0,35,by=0.01),
                                                                                    opt=all_params_coef$opt[curr],
                                                                         a=all_params_coef$a[curr],
                                                                         w=all_params_coef$w[curr],
                                                                         b=all_params_coef$b[curr])))
}

growth_curves_fig = ggplot(built_gr_df %>% dplyr::filter(!is.na(ModeledPoints)))+geom_line(aes(x = Temperature,y=ModeledPoints,group=Strain,
                                  color=factor(Strain)),lwd=1)+ 
    ylim(c(-0.1,1.1)) + scale_color_manual(breaks=as.character((strain_color_frame_harriet %>% dplyr::filter(Strains %in% all_params$Strain))$Strains),
                                        limits=as.character((strain_color_frame_harriet %>% dplyr::filter(Strains %in% all_params$Strain))$Strains),
                                        values=(strain_color_frame_harriet %>% 
                                                dplyr::filter(Strains %in% all_params$Strain))$Colors,
                                           name="Strain")+
    theme_test(base_size=12) + xlab("Temperature (°C)") + ylab("Modeled growth rate (1/day)")
growth_curves_fig

In [ ]:
strain_color_frame_harriet

In [ ]:
norberg_intercept<-function(a,b,width,opt,order) {
    if (order=="one") {
        return(width/2+opt)
        #return((0.75**0.5)*width/2+opt)
    } else {
        return(-width/2+opt)
        #return(-(0.75**0.5)*width/2+opt)
    }
}

In [ ]:
revised_df_width = all_params_coef %>%
    rowwise() %>%
    dplyr::mutate(intercept1=norberg_intercept(a,b,w,opt,"one"),
                  intercept2=norberg_intercept(a,b,w,opt,"two")) %>%
    dplyr::mutate(low_intercept=case_when(Strain=="374" ~ 7,
                                          Strain=="6071" ~ 1,
                                          intercept1>intercept2 ~ intercept2,
                                          TRUE ~ intercept1))%>%
    dplyr::mutate(high_intercept=case_when(intercept1<intercept2 ~ intercept2,
                                          TRUE ~ intercept1)) %>%
    dplyr::mutate(Width=high_intercept-low_intercept)
    

In [ ]:
revised_df_width %>% dplyr::arrange(Width)

In [ ]:
revised_df_2

In [ ]:
width_matrix = ggplot(all_params_coef%>%
                      dplyr::left_join(revised_df_width) %>%
                      dplyr::left_join(revised_df_2) %>%
       dplyr::arrange(desc(Revised_opt)) %>%
       dplyr::mutate(Strain=factor(Strain,levels=unique(Strain))))+ 
    geom_segment(aes(x = 12, xend =Revised_opt,y=Width,
                     yend=Width,color=Strain),pch=21,size=1)+
    geom_segment(aes(y = 12, yend =Width,x=Revised_opt,
                     xend=Revised_opt,color=Strain),pch=21,size=1)+
    geom_point(aes(x = Revised_opt, y =Width,fill=Strain),pch=21,size=7)+theme_bw(base_size=12) + 
    ylab("Estimated thermal width (°C)")+ 
    xlab("Optimum temperature (°C)") + 
    scale_color_manual(breaks=as.character((strain_color_frame_harriet%>%
                                            dplyr::filter(Strains%in%all_params_coef$Strain))$Strains),
                                        limits=as.character((strain_color_frame_harriet%>%
                                            dplyr::filter(Strains%in%all_params_coef$Strain))$Strains),
                                        values=(strain_color_frame_harriet%>%
                                            dplyr::filter(Strains%in%all_params_coef$Strain))$Colors,
                       name="Strain") + 
    scale_fill_manual(breaks=as.character((strain_color_frame_harriet%>%
                                            dplyr::filter(Strains%in%all_params_coef$Strain))$Strains),
                                        limits=as.character((strain_color_frame_harriet%>%
                                            dplyr::filter(Strains%in%all_params_coef$Strain))$Strains),
                                        values=(strain_color_frame_harriet%>%
                                            dplyr::filter(Strains%in%all_params_coef$Strain))$Colors,
                       name="Strain") 
width_matrix

In [ ]:
gr_opt_mat = ggplot(all_params_coef%>%
                      dplyr::left_join(revised_df_width) %>%
                      dplyr::left_join(revised_df_2) %>%
       dplyr::arrange(desc(Revised_opt)) %>%
       dplyr::mutate(Strain=factor(Strain,levels=unique(Strain))))+ 
    #geom_segment(aes(x = 12, xend =Revised_opt,y=Width,
    #                 yend=Width,color=Strain),pch=21,size=1)+
    #geom_segment(aes(y = 12, yend =Width,x=Revised_opt,
    #                 xend=Revised_opt,color=Strain),pch=21,size=1)+
    geom_point(aes(y = Revised_gr_at_opt, x =Width,fill=Strain),pch=21,size=7)+theme_bw(base_size=12) + 
    xlab("Estimated thermal width (°C)")+ 
    ylab("Max growth rate at optimum temperature (1/day)") + 
    scale_color_manual(breaks=as.character((strain_color_frame_harriet%>%
                                            dplyr::filter(Strains%in%all_params_coef$Strain))$Strains),
                                        limits=as.character((strain_color_frame_harriet%>%
                                            dplyr::filter(Strains%in%all_params_coef$Strain))$Strains),
                                        values=(strain_color_frame_harriet%>%
                                            dplyr::filter(Strains%in%all_params_coef$Strain))$Colors,
                       name="Strain") + 
    scale_fill_manual(breaks=as.character((strain_color_frame_harriet%>%
                                            dplyr::filter(Strains%in%all_params_coef$Strain))$Strains),
                                        limits=as.character((strain_color_frame_harriet%>%
                                            dplyr::filter(Strains%in%all_params_coef$Strain))$Strains),
                                        values=(strain_color_frame_harriet%>%
                                            dplyr::filter(Strains%in%all_params_coef$Strain))$Colors,
                       name="Strain") 
gr_opt_mat

In [ ]:
all_params_coef

In [ ]:
write.csv(all_params_coef,"/vortexfs1/omics/alexander/akrinos/2023-Krinos-Ghux-Darwin/code/darwin-processing-scripts/num_coexisting_widths/all_params_experiments_15Dec_fromfinal-code-notebooks.csv")


In [ ]:
max(revised_df_2$Revised_opt)-min(revised_df_2$Revised_opt)

In [ ]:
revised_df_plateau

In [ ]:
plateau_plot = ggplot(revised_df_plateau)+
    geom_segment(aes(x=low_temp,xend=high_temp,y=factor(Strain,
                                                        levels=(revised_df_plateau %>% dplyr::arrange(low_temp))$Strain),
                     yend=factor(Strain,levels=(revised_df_plateau %>% dplyr::arrange(low_temp))$Strain),color=factor(Strain)),lwd=2) + 
    geom_point(aes(x=low_temp,y=factor(Strain,levels=(revised_df_plateau %>% 
                                                      dplyr::arrange(low_temp))$Strain),
                   color=factor(Strain)),size=5) + 
    geom_point(aes(x=high_temp,y=factor(Strain,levels=(revised_df_plateau %>% 
                                                       dplyr::arrange(low_temp))$Strain),
                   color=factor(Strain),color=factor(Strain)),size=5) + 
    xlab("Modeled thermal plateau\n(within 80% GR; °C)")+ 
    ylab("Strain") + theme_bw(base_size=12)+ 
    scale_color_manual(breaks=as.character((strain_color_frame_harriet%>%
                                            dplyr::filter(Strains%in%all_params_coef$Strain))$Strains),
                                        limits=as.character((strain_color_frame_harriet%>%
                                            dplyr::filter(Strains%in%all_params_coef$Strain))$Strains),
                                        values=(strain_color_frame_harriet%>%
                                            dplyr::filter(Strains%in%all_params_coef$Strain))$Colors,
                       name="Strain") 
plateau_plot

In [ ]:
width_plot = ggplot(revised_df_width)+
    geom_segment(aes(x=low_intercept,xend=high_intercept,y=factor(Strain,
                                                        levels=(revised_df_width %>% dplyr::arrange(low_intercept))$Strain),
                     yend=factor(Strain,levels=(revised_df_width %>% dplyr::arrange(low_intercept))$Strain),color=factor(Strain)),lwd=2) + 
    geom_point(aes(x=low_intercept,y=factor(Strain,levels=(revised_df_width %>% 
                                                      dplyr::arrange(low_intercept))$Strain),
                   color=factor(Strain)),size=5) + 
    geom_point(aes(x=high_intercept,y=factor(Strain,levels=(revised_df_width %>% 
                                                       dplyr::arrange(low_intercept))$Strain),
                   color=factor(Strain),color=factor(Strain)),size=5) + 
    xlab("Modeled thermal width (°C)")+ 
    ylab("Strain") + theme_bw(base_size=12)+ 
    scale_color_manual(breaks=as.character((strain_color_frame_harriet%>%
                                            dplyr::filter(Strains%in%all_params_coef$Strain))$Strains),
                                        limits=as.character((strain_color_frame_harriet%>%
                                            dplyr::filter(Strains%in%all_params_coef$Strain))$Strains),
                                        values=(strain_color_frame_harriet%>%
                                            dplyr::filter(Strains%in%all_params_coef$Strain))$Colors,
                       name="Strain") 
width_plot

In [ ]:
(plateau_plot|width_plot) +plot_layout(guides = "collect") & plot_annotation(tag_levels="A")
ggsave("../../figures/supplemental/widths_v_plateau.png",width=10,height=6,units="in")

In [ ]:
revised_df_width %>% dplyr::arrange(Width)

In [ ]:
max(revised_df_width$Width)-min(revised_df_width$Width)

In [ ]:
ALL_EHUX = read.csv('/vortexfs1/omics/alexander/halexander/2020-Ehux/RCC-parsing/EHUX_all_strains.csv')

ALL_EHUX %>% dplyr::select(X,RCC.temperature,Latitude,Longitude)%>%
    dplyr::filter(X %in% c("RCC6856","RCC6071","RCC1212","RCC3963",
                           "RCC874","CCMP1280","RCC4567","RCC914","RCC3492",
                           "CCMP371","RCC1215","CCMP377","RCC1239",
                           "RCC1222","RCC1256","CCMP74","CCMP379","CCMP1516",
                           "CCMP2090","CCMP375"))%>%
    dplyr::bind_rows(data.frame("X"=c("CCMP371","CCMP1280","CCMP375","CCMP377","CCMP1516/2090","CCMP379"),
                                "Latitude"=c(32,-12,32,43,-2.67,50.17),
                                "Longitude"=c(-62,-35,-62,-68,-82.72,-4.25), # deposit date is used if sampling unavailable
                                "SamplingYear"=c(1987,1990,1967,1988,1992,NA),
                                "SamplingMonth"=c(6,10,2,9,9,NA)))

other_isolation_dates = data.frame("X"=c("RCC874","RCC914","RCC1212","RCC1215","RCC1222",
                                         "RCC1239","RCC1256","RCC3492","RCC3963","RCC6071"),
                                   "SamplingYear"=c(2004,2004,2000,2001,1998,2002,1999,2011,
                                                    2011,2018),
                                   "SamplingMonth"=c(11,10,9,2,7,4,7,12,10,3),
                                   "SamplingDepth"=c(5,10,3,NA,20,NA,NA,0,NA,10),
                                   "SamplingTemp"=c(NA,27.78,NA,NA,NA,NA,NA,NA,NA,NA))

sample_t_frame = ALL_EHUX %>% dplyr::select(X,RCC.temperature,Latitude,Longitude)%>%
    dplyr::filter(X %in% c("RCC6856","RCC6071","RCC1212","RCC3963",
                                                   "RCC874","CCMP1280","RCC4567","RCC914","RCC3492",
                                                   "CCMP371","RCC1215","CCMP377","RCC1239",
                                                   "RCC1222","RCC1256","CCMP74","CCMP379","CCMP1516",
                                                   "CCMP2090","CCMP375"))%>%
    dplyr::left_join(other_isolation_dates,by=c("X"))%>%
    dplyr::bind_rows(data.frame("X"=c("CCMP371","CCMP1280","CCMP375","CCMP377","CCMP1516","CCMP2090","CCMP379"),
                                "Latitude"=c(32,-12,32,43,-2.67,-2.67,50.17),
                                "Longitude"=c(-62,-35,-62,-68,-82.72,-82.72,-4.25), # deposit date is used if sampling unavailable
                                "SamplingYear"=c(1987,1990,1967,1988,1992,1992,NA),
                                "SamplingMonth"=c(6,10,2,9,9,9,NA),
                                "SamplingTemp"=c(NA,NA,NA,NA,NA,NA,NA)))

final_merge_df=sample_t_frame %>% separate(X,into=c("RCC","StrainRCC"),sep="RCC") %>%
    tidyr::separate(RCC,into=c("CCMP","StrainCCMP"),sep="CCMP") %>%
    dplyr::mutate(Strain=as.numeric(case_when(is.na(StrainCCMP)~StrainRCC,
                                   StrainCCMP=="1516/2090"~"1516",
                                   TRUE ~ StrainCCMP))) %>%
    dplyr::bind_rows(data.frame("Strain"=as.numeric(c("371","1280","375","377",
                       "1516","2090","379","4567","371","374")),
                "Latitude"=c(32,-12,32,43,-2.67,-2.67,50.1669,-11.48,32,42.5),
                "Longitude"=c(-62,-35,-62,-68,-82.72,-82.72,-4.2504,-25.05,-62,-69)))

In [ ]:
print_summaries <- function(dataframe_in,column) {
    curr_data=dataframe_in[column]
    print(paste("Min of column",column,"is",round(min(curr_data),2),sep=" "))
    print(paste("Max of column",column,"is",round(max(curr_data),2),sep=" "))
    print(paste("Range of column",column,"is",round(max(curr_data)-min(curr_data),2),sep=" "))
}
print_summaries(all_params_coef,"opt")
print_summaries(all_params_coef,"w")

In [ ]:
latitude_opt=ggplot(all_params_coef%>%
                      dplyr::left_join(revised_df_width) %>%
                      dplyr::left_join(revised_df_2)%>% dplyr::left_join(final_merge_df)) + geom_point(aes(y=Revised_opt,x=Latitude,fill=factor(Strain)),pch=21,color="black",
                                                                                                       size=3)+ 
    scale_fill_manual(breaks=as.character((strain_color_frame_harriet %>% dplyr::filter(Strains %in% all_params$Strain))$Strains),
                                        limits=as.character((strain_color_frame_harriet %>% dplyr::filter(Strains %in% all_params$Strain))$Strains),
                                        values=(strain_color_frame_harriet %>% 
                                                dplyr::filter(Strains %in% all_params$Strain))$Colors,
                      name="Strain")+
    theme_bw(base_size=12) + ylab("Thermal optimum (°C)") + xlab("Latitude of original isolation")+
   geom_smooth(data=all_params_coef%>%
                      dplyr::left_join(revised_df_width) %>%
                      dplyr::left_join(revised_df_2) %>% dplyr::left_join(final_merge_df),
               method="lm", aes(x=Latitude,y=Revised_opt,color="Exp Model"), color="black",
               formula= (y ~  x + I(x^2)), 
               se=FALSE, linetype = 1)
latitude_opt

In [ ]:
ggplot(all_params_coef %>% dplyr::left_join(final_merge_df) %>% dplyr::left_join(revised_df_width)) + 
    geom_point(aes(x=Latitude,y=Width,color=factor(Strain)),size=4)+ 
    scale_color_manual(breaks=as.character((strain_color_frame_harriet %>% dplyr::filter(Strains %in% all_params$Strain))$Strains),
                                        limits=as.character((strain_color_frame_harriet %>% dplyr::filter(Strains %in% all_params$Strain))$Strains),
                                        values=(strain_color_frame_harriet %>% 
                                                dplyr::filter(Strains %in% all_params$Strain))$Colors,
                      name="Strain")+
    theme_bw(base_size=12) + xlab("Latitude") +ylab("Modeled temperatures (°C) within 90%\nof growth rate at optimum temp") 


In [ ]:
best_gr_frame=data.frame()
for (curr in 1:nrow(all_params_coef)) {
    best_gr_frame=best_gr_frame %>% dplyr::bind_rows(data.frame(Strain=all_params_coef$Strain[curr],
                                                            Temperature = all_params_coef$opt[curr],
                                                            ModeledPoints = nbcurve(all_params_coef$opt[curr],
                                                                                    opt=all_params_coef$opt[curr],
                                                                         a=all_params_coef$a[curr],
                                                                         w=all_params_coef$w[curr],
                                                                         b=all_params_coef$b[curr]),
                                                            w = all_params_coef$w[curr],
                                                            opt = all_params_coef$opt[curr]))
}
best_gr_frame=best_gr_frame%>%dplyr::left_join(revised_df_width) %>%
    dplyr::left_join(revised_df_plateau)

In [ ]:
colnames(best_gr_frame)

In [ ]:
cor.test(best_gr_frame$topt,best_gr_frame$Revised_plateau, method="kendall")

In [ ]:
cor.test(best_gr_frame$gr_opt,best_gr_frame$Revised_plateau, method="kendall")

In [ ]:
cor.test(best_gr_frame$Revised_plateau,best_gr_frame$Width, method="kendall")

In [ ]:
cor.test(best_gr_frame$ModeledPoints,best_gr_frame$w, method="kendall")

In [ ]:
cor.test(best_gr_frame$ModeledPoints,best_gr_frame$opt, method="kendall")

In [ ]:
all_params_coef%>%
                      dplyr::left_join(revised_df_width) %>%
                      dplyr::left_join(revised_df_2)

In [ ]:
summary(lm(formula= (Revised_opt ~  Latitude + I(Latitude^2)),data=all_params_coef%>%
                      dplyr::left_join(revised_df_width) %>%
                      dplyr::left_join(revised_df_2)%>% dplyr::left_join(final_merge_df)))

In [ ]:
summary(lm(formula= (Width ~  Latitude + I(Latitude^2)),data=all_params_coef %>% 
           dplyr::left_join(final_merge_df) %>%
           dplyr::left_join(revised_df_width)))

In [ ]:
(growth_curves_fig|eppley_comp|latitude_opt)+ plot_layout(guides = "collect") & plot_annotation(tag_levels = 'A')
ggsave("/vortexfs1/omics/alexander/akrinos/2023-Krinos-Ghux-Darwin/figures/main/Figure2_10Feb24.pdf",
      width=12,height=4,units="in")

In [ ]:
(growth_curves_fig|eppley_comp)/(latitude_opt|width_matrix)+ plot_layout(guides = "collect") & plot_annotation(tag_levels = 'A')
ggsave("/vortexfs1/omics/alexander/akrinos/2023-Krinos-Ghux-Darwin/figures/main/Figure2_25Nov23.pdf",
      width=12,height=9,units="in")

In [ ]:
(growth_curves_fig|eppley_comp)/(latitude_opt|gr_opt_mat)+ plot_layout(guides = "collect") & plot_annotation(tag_levels = 'A')
ggsave("/vortexfs1/omics/alexander/akrinos/2023-Krinos-Ghux-Darwin/figures/main/Figure2_15Dec23.pdf",
      width=10,height=9,units="in")

In [ ]:
width_matrix
ggsave("/vortexfs1/omics/alexander/akrinos/2023-Krinos-Ghux-Darwin/figures/supplemental/Figure2_width_supp_15Dec23.pdf",
      width=5,height=4.5,units="in")
ggsave("/vortexfs1/omics/alexander/akrinos/2023-Krinos-Ghux-Darwin/figures/supplemental/Figure2_width_supp_15Dec23.png",
      width=5,height=4.5,units="in")